In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator
import os

In [3]:
zip_path = '/content/drive/MyDrive/datasets/dataset.zip'
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/brain-tumor-dataset')

In [4]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout
from sklearn.metrics import accuracy_score

import ipywidgets as widgets
import io
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
testing_set = test_datagen.flow_from_directory(
    '/content/brain-tumor-dataset/dataset/Testing',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True)

Found 394 images belonging to 4 classes.


In [5]:
X_train = []
Y_train = []
image_size = 224
labels = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
for i in labels:
    folderPath = os.path.join('/content/brain-tumor-dataset/dataset/Training',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        Y_train.append(i)

for i in labels:
    folderPath = os.path.join('/content/brain-tumor-dataset/dataset/Testing',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        Y_train.append(i)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [6]:
X_train,Y_train = shuffle(X_train,Y_train,random_state=101)
X_train.shape

(3264, 224, 224, 3)

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X_train,Y_train,test_size=0.1,random_state=101)

In [8]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train=y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test=y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [82]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import cv2
from sklearn.metrics import accuracy_score

cnn_model = load_model("/content/drive/MyDrive/datasets/BrainTumorCNN.h5")
resnet_model = load_model("/content/drive/MyDrive/datasets/BrainTumorResnet50.h5")


In [90]:
cnn_predictions = cnn_model.predict(X_test)
resnet_predictions = resnet_model.predict(X_test)

y_true = np.argmax(y_test, axis=1)

cnn_labels = np.argmax(cnn_predictions, axis=1)
resnet_labels = np.argmax(resnet_predictions, axis=1)

cnn_accuracy = accuracy_score(y_true, cnn_labels)
resnet_accuracy = accuracy_score(y_true, resnet_labels)

print("CNN Accuracy:", cnn_accuracy)
print("ResNet Accuracy:", resnet_accuracy)

weight_cnn = 0.5
weight_resnet = 0.5
ensemble_predictions = weight_cnn * cnn_predictions + weight_resnet * resnet_predictions

ensemble_labels = np.argmax(ensemble_predictions, axis=1)

ensemble_accuracy = accuracy_score(y_true, ensemble_labels)
print("Ensemble Accuracy:", ensemble_accuracy)

11/11 [==============================] - 1s 95ms/step
CNN Accuracy: 0.9021406727828746
ResNet Accuracy: 0.9113149847094801
Ensemble Accuracy: 0.9296636085626911


In [99]:
img = cv2.imread('/content/brain-tumor-dataset/dataset/Testing/meningioma_tumor/image(120).jpg')
img = cv2.resize(img, (224, 224))
img_array = np.array(img)
img_array = img_array.reshape(1, 224, 224, 3)

cnn_prediction = cnn_model.predict(img_array)
resnet_prediction = resnet_model.predict(img_array)

weight_cnn = 0.5
weight_resnet = 0.5
ensemble_prediction = weight_cnn * cnn_prediction + weight_resnet * resnet_prediction

final_label = np.argmax(ensemble_prediction, axis=1)
print(np.argmax(cnn_prediction, axis=1))
print(np.argmax(resnet_prediction, axis=1))
print("Ensemble Prediction:", final_label)

1/1 [==============================] - 0s 55ms/step
[1]
[1]
Ensemble Prediction: [1]
